In [39]:
import google.generativeai as genai
import ast
import os
import json
import PIL.Image
from selenium.webdriver.common.by import By

# Load environment variables
from dotenv import load_dotenv
from selenium_functions import open_browser

load_dotenv()

# Convert the GEMINI_API_KEYS string from environment variables to a list
GEMINI_API_KEYS = os.environ.get("GEMINI_API_KEYS")
KEY_LIST = ast.literal_eval(GEMINI_API_KEYS)

# Randomly shuffle the list of API keys
import random
random.shuffle(KEY_LIST)

# Global index to keep track of the current key
current_api_key_index = 0

In [40]:
from lxml import html


def extract_elements_by_xpath(html_string, xpath_selector):
    # Parse the HTML
    tree = html.fromstring(html_string)

    # Apply the XPath selector
    elements = tree.xpath(xpath_selector)

    # Return a list of outer HTML for each element
    return str([html.tostring(element).decode("utf-8") for element in elements] + [xpath_selector])

In [41]:
# goal: "type": "src", "selector": "some url I think"
system_prompt_interpret = """
You are a web browser navigation assistant that trims and scrapes relevant portions of the UI for a user. Relevant is defined as the portion of the UI that the user requests for.
Only return selectors or images that are relevant to the user's request.

Whenever a user requests something, you will return the xpath selector or the src attribute of an image that returns the path to the specific file the image is stored in within the client file of the website.
Ensure that all paths end with the file extension of the image (examples are .jpg, .png, .gif, etc.)

If the request requires multiple choices, return ALL RELEVANT selectors that contains the UI that will enable the user to choose the choice themselves.
For example, if there is a container containing two buttons, and it is ambiguous which button the user is interested in, return a selector to the container instead of one of the buttons only.

Output your result in the following format and output as many selectors as necessary. Ensure that the output is a JSON object and that there is a diversity of file paths aligned to the specific types of each image:
[
    {
        "type": xpath
        "selector": selector
    },
    {
        "type": src
        "selector": the src attribute of the image represented as the route to the image inside of the client file of the website
    },
    ...
]
"""

system_prompt_generate = """
You are a web browser navigation assistant that generates a user interface for a user to interact with.
You will be given DOM elements from another web browser navigation assistant that trims and scrapes relevant portions of the UI for a user.

Your task is to generate valid HTML strings that can be rendered in a browser, specifically focusing on interactive elements such as buttons and text fields. 
Please use TailwindCSS for styling. Use actual hex colors for the colors, do not use TailwindCSS classes for colors.

Each element should only have two attributes:
- class: a string of classes separated by spaces, for TailwindCSS styling
- special-id: the XPath or id selector that was given to you, which will be used for identifying the element during interactions
- type: a string that is either 'text', 'button', 'input', or 'img'

Only output images if they are contained in the DOM elements that were given to you.

Output your result in the following format:
<div class='container classes here'>
    <div type='text' class='input classes here'>
        <!-- Additional content here -->
    </div>
    <div type='button' class='button classes here' special-id='button selector here'">
        <!-- Additional content here -->
    </div>
    <input type='input' class='input classes here' special-id='input selector here'>
    <img type='img' class='img classes here' src='image source here'>
</div>
"""

In [42]:
def cycle_api_key():
    global current_api_key_index
    if current_api_key_index >= len(KEY_LIST) - 1:
        current_api_key_index = 0
    else:
        current_api_key_index += 1
    return KEY_LIST[current_api_key_index]


def generate_content_with_cycling_keys(prompt, system_prompt, image=None):
    global current_api_key_index
    # Get the current API key and cycle to the next one for future requests
    api_key = cycle_api_key()

    # Configure the generative AI model with the new API key
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(
        "gemini-1.5-pro-latest",
        generation_config=genai.GenerationConfig(
            max_output_tokens=8000,
            temperature=0,
        ),
        system_instruction=system_prompt,
    )

    # Generate content using the provided prompt
    if image is None:
        response = model.generate_content(prompt)
    else:
        response = model.generate_content([prompt, image])
    return response.text

In [43]:
# Load website.html into a string
with open('website.html', 'r') as file:
    html = file.read()

user_prompt = f"""
current_page: {html}

user: Show me the deals
"""

# Generate content using the prompt and the website HTML
response = generate_content_with_cycling_keys(user_prompt, system_prompt_interpret)
if "```json" in response:
    response = response.split("```json")[1].split("```")[0]
obj = json.loads(response)
obj

[{'type': 'xpath', 'selector': "//div[@data-quid='mix-and-match-deal']"},
 {'type': 'xpath', 'selector': "//div[@data-quid='perfect-combo-deal']"},
 {'type': 'xpath', 'selector': "//div[@data-quid='carryout-deal']"},
 {'type': 'src',
  'selector': '/static/1.88.2/images/tiles/mixAndMatchDeal/hero.webp'},
 {'type': 'src',
  'selector': '/static/1.88.2/images/tiles/perfectComboDeal/side.webp'},
 {'type': 'src',
  'selector': '/static/1.88.2/images/tiles/carryoutDeal/side.webp'}]

In [44]:
dom_elements = ""
for element in obj:
    if element['type'] == 'xpath':
        dom_elements += extract_elements_by_xpath(html, element["selector"])
        dom_elements += "\n"
    elif element['type'] == 'id':
        pass
    else:
        dom_elements += f"src: {element['selector']}\n"

In [45]:
print(dom_elements)

['<div data-quid="mix-and-match-deal" dpz-track-evt-name="mix-and-match-deal-hero" class="clickable-card hero css-1gtjg14"><div class="css-15tsjqp"><h2 class="css-1dzbqx5"><span class=" css-1uwlllm" data-quid="mix-and-match-deal-flag">Mix &amp; Match Deal</span></h2></div><div class="css-73v5jz"><p class=" css-7ey74b" data-quid="mix-and-match-deal-headline1">Choose Any 2 or More</p></div><div class="css-jz9tyt"><svg aria-hidden="true" focusable="false" viewbox="0 0 0 0" xmlns="http://www.w3.org/2000/svg" class="css-1snrpgp"><filter id="price-stroke"><femorphology in="SourceGraphic" operator="dilate" radius="2" result="expand"></femorphology><feflood flood-color="#e31837"></feflood><fecomposite in2="expand" operator="in" result="expand"></fecomposite><femerge><femergenode in="expand"></femergenode><femergenode in="SourceGraphic"></femergenode></femerge></filter></svg><span data-quid="mix-and-match-deal-price-formatted" style="border: 0px; clip: rect(0px, 0px, 0px, 0px); height: 1px; mar

In [46]:
design = PIL.Image.open("design.png")
response = generate_content_with_cycling_keys(
    "Use the attached branding guide to style the output. Use the typographies and hex codes defined in the image. Do not use Tailwind Classes\nBase Url for images (if any): https://dominos.com" + dom_elements, system_prompt_generate, image=design
)

In [47]:
print(response)

```html
<div class='container flex flex-col gap-4 p-8'>
    <div type='text' class='text-3xl font-bold' style='font-family: "Inter Bold"; color: #2B2B2B;'>
        Mix &amp; Match Deal
    </div>
    <div type='text' class='text-xl font-semibold' style='font-family: "Inter Semi-bold"; color: #2B2B2B;'>
        Choose Any 2 or More
    </div>
    <div type='text' class='text-4xl font-bold' style='font-family: "Inter Bold"; color: #E31837;'>
        $6.99 each
    </div>
    <div type='text' class='text-sm' style='font-family: "Inter Medium"; color: #9F03FE;'>
        California pricing higher
    </div>
    <div type='text' class='text-xs' style='font-family: "Inter Medium"; color: #9F03FE;'>
        2-Item Minimum. Bone-in Wings, Bread Bowl Pasta, and Handmade Pan Pizza will cost extra. Prices, delivery area, and charges may vary by store. Delivery orders are subject to each local store's delivery charge. 
    </div>
    <div type='button' class='rounded-md px-4 py-2 text-white' style=